# **Exploração e análise de dados de crédito com SQL**

#  **Os dados:**

Os dados representam informações de clientes de um banco e contam com as seguintes colunas:
* idade = idade do cliente
* sexo = sexo do cliente (F ou M)
* dependentes = número de dependentes do cliente
* escolaridade = nível de escolaridade do clientes
* estado civil = situação conjugal dos clientes
* salario_anual = faixa salarial do cliente
* tipo_cartao = tipo de cartao do cliente
* qtd_produtos = quantidade de produtos comprados nos últimos 12 meses
* iteracoes_12m = quantidade de iterações/transacoes nos ultimos 12 meses
* meses_inativo_12m = quantidade de meses que o cliente ficou inativo
* limite_credito = limite de credito do cliente
* valor_transacoes_12m = valor das transações dos ultimos 12 meses
* qtd_transacoes_12m = quantidade de transacoes dos ultimos 12 meses

A tabela foi criada no AWS Athena junto com o S3 Bucket com uma versão dos dados disponibilizados em:  https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/credito.csv

# **Criação da tabela e algumas queries**

```sql
CREATE EXTERNAL TABLE IF NOT EXISTS default.credito (
  `idade` int,
  `sexo` string,
  `dependentes` int,
  `escolaridade` string,
  `estado_civil` string,
  `salario_anual` string,
  `tipo_cartao` string,
  `qtd_produtos` bigint,
  `iteracoes_12m` int,
  `meses_inativo_12m` int,
  `limite_credito` float,
  `valor_transacoes_12m` float,
  `qtd_transacoes_12m` int
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
  'serialization.format' = ',',
  'field.delim' = ','
) LOCATION <sua-localizacao>
TBLPROPERTIES ('has_encrypted_data'='false');
```



# **Exploração de dados:**
A primeira fase da análise é entender a nossa base de dados. Portanto, vamos iniciar a exploração de dados:


**Qual a quantidade de informações temos na nossa base de dados?**

**Query-1:** SELECT count(*) as qtd_linhas FROM credito

![Resposta](https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/qtd_linhas.JPG?raw=true)


**nota**.: A base de dados do link acima contém mais linhas do que a seleção utilizada. Você pode utilizar todas as linhas ou considerar apenas uma fração dos dados. Na prática, quanto maior a quantidade de dados utilizada, mais confiável é a análise! Mas, existem limites computacionais e financeiros, no qual a redução de dados para análise para fins de estudo se torna interessante.

**Como são os dados:**

**Query-2:** SELECT * FROM credito LIMIT 10;
![](https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/1-select..JPG?raw=true)

> É possível reparar que existem algumas informações nulas na tabela (valor na), vamos olhar mais de perto para os valores de cada coluna!

**Quais os tipos de cada dado:**

**Query-3:** DESCRIBE credito

![Resposta](https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/describe.JPG?raw=true)

> Agora, podemos compreender quais são os tipos de dados, vamos analisar mais atentamente para as varíaveis que não são numéricas.

**Quais são os sexos?**

**Query-4:** SELECT DISTINCT sexo FROM credito

![](https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/4-sexo.JPG?raw=true)

**Quais são os tipos de escolaridade disponíveis no dataset?**

**Query-5:** SELECT DISTINCT escolaridade FROM credito

![Resposta](https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/3-distinct.JPG?raw=true)

> Os dados contam com vários níveis de escolaridade e é possível perceber que temos valores nulos (na) no dataset, trataremos isso mais tarde!

Será que temos mais valores nulos?

**Quais são os tipos de estado_civil disponíveis no dataset?**

**Query-6:** SELECT DISTINCT estado_civil FROM credito

![](https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/estado_civil.JPG?raw=true)

> Novamente encontramos valores nulos nos dados de estado civil!

**Quais são os tipos de salario_anual disponíveis no dataset?**

**Query-7:** SELECT DISTINCT salario_anual FROM credito

![](https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/salario_anual.JPG?raw=true)

> Os salários nesse dataset não estão expostas com o valor exato que o cliente ganha. O que está informado é a faixa salarial de cada e, também, contêm dados nulos.

**Quais são os tipos de cartão disponíveis no dataset?**

**Query-8:** SELECT DISTINCT tipo_cartao FROM credito

![](https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/tipo_cartoes.png?raw=true)

> Aqui vemos que não há necessidade de tratar valores nulos.

# **Análise de dados**

Uma vez que exploramos o dados e buscamos entender quais são as informações que temos no nosso banco de dados, podemos analisar as informações para buscar entender o que está acontecendo no banco de dados. Vamos fazer iniciar as perguntas:  

**Nesse banco de dados, quantos clientes temos de cada faixa salarial?**

**Query-9:** select count(*), salario_anual from credito group by salario_anual

![](https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/Qtde_faixa%20salarial.PNG?raw=true)
![](https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/Graf_Clientes%20por%20Faixa%20Salarial%20anual.PNG?raw=true)

> A maioria dos clientes dessa base de dados possui um renda menor que 40K e existem 235 clientes que não informaram ou não consta a faixa salarial. De certa forma, pode ser interessante para a empresa focar nesse público de mais baixa renda.

**Qual quantidade de clientes por cartão?**

**Query-10:** select count(*) as qtd_clientes, tipo_cartao from credito group by tipo_cartao

![](https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/7-qtde_cliente_cartao.JPG?raw=true)

> Verificamos que o cartão blue possui a maior quantidade de clientes (2453), onde pode ser interessante focar nesse público e a minoria (02) possui um cartão platinum.


**Nesse banco de dados, quantos clientes são homens e quantos são mulheres?**

**Query-11:** select count(*), sexo from credito group by sexo

![](https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/Qtde_sexo%202.PNG?raw=true)
![](https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/Graf_Sexo%202.PNG?raw=true)


> A maioria dos clientes desse banco são homens e à partir do arquivo csv extraído dos dados é possível gerar um gráfico em pizza, para melhor visualização da proporção de masculino/feminino.

**Queremos focar o nosso marketing de maneira adequada para nossos clientes, qual será a idade deles?**

**Query-12:** select avg(idade) as media_idade, min(idade) as min_idade, max(idade) as max_idade, sexo from credito group by sexo

![](https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/Segmentar%20por%20idade.PNG?raw=true)

> Por meio dessa análise não foi possível extrair nenhuma informação relevante. A menor idade dos dois sexos é a mesma e a média é muito similar. A única diferença é a idade máxima, mas é um fato quase irrelevante pois a diferença não é tão gritante.

**Qual a maior e menor transação dos clientes?**

**Query-13:** select min(valor_transacoes_12m) as gasto_minimo_cliente, max(valor_transacoes_12m) as gasto_maximo_cliente from credito

![](https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/11-maior_menor_transacao.JPG?raw=true)

![](https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/Graf_Transa%C3%A7%C3%A3o%20min%2Bmax%20anual.PNG?raw=true)

> Nesse banco de dados temos soma de transações em 12 meses que variam de 510.16 à 4776.58

**Quais as características dos clientes que possuem os maiores créditos?**

**Query-14:** select max(limite_credito) as limite_credito, escolaridade, tipo_cartao, sexo from credito where escolaridade != 'na' and tipo_cartao != 'na' group by escolaridade, tipo_cartao, sexo order by limite_credito desc limit 10

![](https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/12-maiores_creditos.JPG?raw=true)

> Não parece haver um impacto da escolaridade no limite. O limite mais alto é oferecido para um homem sem educação formal. O cartão também parece não estar relacionado com a escolaridade nem com o limite. Dentre os maiores limites, encontramos clientes com cartão: gold, silver, platinum e blue

**Quais as características dos clientes que possuem os menores créditos?**

**Query-15:** select min(limite_credito) as limite_credito, escolaridade, tipo_cartao, sexo from credito where escolaridade != 'na' and tipo_cartao != 'na' group by escolaridade, tipo_cartao, sexo order by limite_credito desc limit 10

![](https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/13-menores_creditos.JPG?raw=true)

> Nessa amostra, verificamos que não parece ter impacto da escolaridade com o limite de crédito.  Pois, um cliente com escolaridade 'mestrado' e cartão 'gold' do sexo 'F' tem o menor limite de crédito. Podemos pressumir que o cartão não está relacionado com a escolaridade nem com o limite.

**Quem gasta mais homens ou mulheres?**

**Query-16:** select max(valor_transacoes_12m) as maior_valor_gasto, avg(valor_transacoes_12m) as media_valor_gasto, min(valor_transacoes_12m) as min_valor_gasto, sexo from credito group by sexo

![](https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/14-quem_gasta_mais.JPG?raw=true)

> Apesar da diferença nos limites, os gastos de homens e mulheres são similares.

**O estado civil e o sexo influencia no valor gasto?**

**Query-17:** select max(valor_transacoes_12m) as maior_valor_gasto, avg(valor_transacoes_12m) as media_valor_gasto, min(valor_transacoes_12m) as min_valor_gasto, sexo, estado_civil
from credito
where estado_civil != 'na'
group by sexo, estado_civil
order by avg(valor_transacoes_12m) desc

![](https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/15-estadoCivil_sexo_gasto.JPG?raw=true)

> Sim! O cliente do estado civil 'feminino e solteira' apresenta maior valor gasto. Mas, a amostra também moostra que tanto os homens quanto as mulheres dos estado_civil (solteiro, casado ou divorcidado), possuem uma média de valor gasto muito similar, em que as vezes a diferença entre eles não ultrapassa 100.0.

**O salário impacta no limite?**

**Query-18:** select avg(qtd_produtos) as qts_produtos, avg(valor_transacoes_12m) as media_valor_transacoes, avg(limite_credito) as media_limite, sexo, salario_anual from credito where salario_anual != 'na' group by sexo, salario_anual order by avg(valor_transacoes_12m) desc

![](https://github.com/murilovalenso/Analise-de-dados-de-credito-com-SQL/blob/main/16-salarioxlimite.jpg?raw=true)

> SIM! As pessoas que tem menor faixa salarial também apresentam menor limite de credito!

# **Conclusão**

Essas foram algumas análises extraídas do dataset de crédito.


Alguns insights interessantes:

* a maior parte dos clientes possui renda até 40K
* a maior parte dos clientes é masculino!
* a escolaridade não parece influenciar no limite nem no tipo do cartão
* o sexo e estado civil não parece influenciar no limite do cartão
* os clientes com maiores limites são em sua maioria homens
* os clientes com menores limites são em sua maioria mulheres
* dentre os menores limites não há presença de cartão platinum
* a faixa salarial impacta diretamente no limite de crédito
* nao existem clientes com salário anual acima de 60K do sexo feminino
* as mulheres e solteiras apresentam maior valor gasto


Uma exploração maior dos dados pode explicar porque as mulheres tem menor crédito. Isso também pode ser um problema cultural que pode ser repensado!